In [ ]:
!pip install librosa
%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib.pyplot as plt
import IPython.display as ipd
import torch
from torch import nn
from scipy import signal
import glob
import librosa
import librosa.display
import time
import subprocess
import os
from scipy.io import wavfile
!wget https://github.com/digitalmusicprocessing/Week13_SIREN/raw/refs/heads/main/doves.wav

In [ ]:
class MLP(nn.Module):
    def __init__(self, depth=3, n_input=1, n_output=1, n_units=512, normalize_weights=True, w0=5000):
        super(MLP, self).__init__()
        ## TODO: Fill this in
        
    def forward(self, x):
        ## TODO: Fill this in
        pass

    def get_num_parameters(self):
        total = 0
        for p in self.parameters():
            total += np.prod(p.shape)
        return total

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') 
print("Device: ", device)

sr = 44100
y, _ = librosa.load("doves.wav", sr=sr)
y = y[0:sr*4]
N = y.size
y = torch.from_numpy(y).to(device)
t = torch.linspace(0, 1, N)
t = t.view((N, 1)).to(device)

In [ ]:
model = MLP(depth=3, n_input=1, n_output=1, n_units=64, normalize_weights=True, w0=5000).to(device)
print("Compression:", model.get_num_parameters()/N)

plt.plot(model(t).detach().cpu()[0:10000])
ipd.Audio(model(t).detach().cpu().flatten(), rate=sr)

In [ ]:
n_iters = 5000
samples_per_iter = sr
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
losses = []

yest = model(t)

for i in range(n_iters):
    optimizer.zero_grad()
    yest = model(t)
    loss = torch.mean(torch.abs(y-yest.flatten()))
    losses.append(loss.item())
    loss.backward()
    optimizer.step()
    if i%100 == 0:
        ipd.clear_output()
        print(i, loss)

plt.plot(losses)